<a href="https://colab.research.google.com/github/federico-caria/selectToAutocomplete/blob/master/test_archive_esoteric_journals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [3]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 17.7 MB/s eta 0:00:00


In [4]:
import PyPDF2

In [11]:
import io

In [12]:
import subprocess
import shutil

In [17]:
!pip install pdfminer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 73.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.6 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140089 sha256=8b1c6c9a73e5020d7502eeec50dd6d942ba8f9429d362aa4742c4eda8ed5ef3c
  Stored in directory: /root/.cache/pip/wheels/d1/aa/48/370f83a970d62355a2a47d2d640094a64eea932c22edae1891
Successfully built pdfminer


In [18]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [29]:
def extract_metadata(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        return pdf_reader.metadata

metadata = extract_metadata('/content/UR_1928.pdf')
print(metadata)


{'/CreationDate': 'D:20171011220545Z', '/Creator': 'Preview', '/ModDate': "D:20171012121224-04'00'", '/Producer': 'ABBYY FineReader 12', '/Title': 'Gruppo di UR - UR (1928) 2.vol. Ed Tilopa.pdf'}


In [50]:

def split_pdf(pdf_path, chunk_size=100):
    # Create a folder to store the split files if it doesn't exist
    if not os.path.exists('ur-journal'):
        os.makedirs('ur-journal')
        
    # Open the PDF file and get metadata
    with open(pdf_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        metadata = pdf_reader.metadata
        with open('ur-journal/metadata.txt', 'w') as metadata_file:
            metadata_file.write(str(metadata))

        # Split the PDF into smaller chunks
        total_pages = len(pdf_reader.pages)
        for i in range(0, total_pages, chunk_size):
            start = i
            end = min((i + 1) * chunk_size, total_pages)
            output_path = f"ur-journal/{os.path.splitext(os.path.basename(pdf_path))[0]}_{i+1}.pdf"
            with open(output_path, 'wb') as output:
                pdf_writer = PyPDF2.PdfWriter()
                for page_num in range(start, end):
                    page = pdf_reader.pages[page_num]
                    pdf_writer.add_page(page)
                pdf_writer.write(output)
                
                # Convert the PDF file to plain text
                text_output_path = f"ur-journal/{os.path.splitext(os.path.basename(output_path))[0]}.txt"
                with open(text_output_path, 'w') as text_output:
                    for page_num in range(end-start):
                        page = pdf_writer.pages[page_num]
                        text = page.extract_text()
                        text_output.write(text)
                
                # Save metadata for the split file
                with open(f"ur-journal/{os.path.splitext(os.path.basename(output_path))[0]}_metadata.txt", 'w') as metadata_file:
                    metadata_file.write(str(metadata))

# Example usage
split_pdf('/content/UR_1928.pdf')
